In [ ]:
from pprint import pprint
import copy




In [76]:
#register('get_raw', {in: {}, out: {raw: raw_meta}} 
def get_raw(obj, opt=None):
    opt_default={'debug': True, 'eer':2}
    if not opt is None:
        opt = {**opt_default, **opt}
    else: opt = opt_default
    
    uuid = '0001'
    hist = {}
    if opt['debug']: hist['data'] = copy.deepcopy(obj['data'])
    if opt['debug']: hist['map'] = copy.deepcopy(obj['map'])

    if not 'raw' in obj['data']: obj['data']['raw'] = []
    index = len(obj['data']['raw'])
    obj['data']['raw'].append([1,2,3,4])

    if not 'raw' in obj['meta']: obj['meta']['raw'] = []
    obj['meta']['raw'].append({'type': "list", 'dim': 1, 'name': "RawVoltage", 'label': {'de':"Spannung"}, 'quant': "qudt:Voltage", 'unit':"qudt:mV", 'test':{'nested': "value"}})

    hist['func'] = {'func_id': uuid, 'name': "get_raw", 'time':"...", 'opt':opt}
    obj['hist'].append(hist)
    
    return obj
    
def get_raw2(obj, opt=None):
    opt_default={'debug': True, 'eer':2}
    if not opt is None:
        opt = {**opt_default, **opt}
    else: opt = opt_default
    
    uuid = '0002'
    hist = {}
    if opt['debug']: hist['data'] = copy.deepcopy(obj['data'])
    if opt['debug']: hist['map'] = copy.deepcopy(obj['map'])

    if not 'raw2' in obj['data']: obj['data']['raw2'] = []
    index = len(obj['data']['raw2'])
    obj['data']['raw2'].append([1,2,3,4])

    if not 'raw2' in obj['meta']: obj['meta']['raw2'] = []
    obj['meta']['raw2'].append({'type': "list", 'dim': 1, 'name': "RawVoltage", 'label': {'de':"Spannung"}, 'quant': "qudt:Voltage", 'unit':"qudt:mV"})

    hist['func'] = {'func_id': uuid, 'name': "get_raw2", 'time':"...", 'opt':opt}
    obj['hist'].append(hist)

    return obj


In [ ]:
map = {
	x: {data: {index:, meta:{name:"RawVoltage"}}},
	label: {meta: {key: 'label', lang: 'de'}},
    debug: False(obj) => obj['data']['raw'] = 0 
}

In [ ]:
register('plot', {in: {x: {type: "numpy"}, label {type:"string"}}, out: {raw}} 
func plot(obj, opt={debug: true}) {
...33
}

In [ ]:
obj.exec({
    f: "raw_data",
    x: {data: {index:, meta:{name:"RawVoltage"}}},
    label: {meta: {key: 'label', lang: 'de'}},
    debug: False(obj) => obj['data']['raw'] = 0 
}).exec({
    x: {data: {index:, meta:{name:"RawVoltage"}}},
    label: {meta: {key: 'label', lang: 'de'}},
    debug: False(obj) => obj['data']['raw'] = 0 
})

In [ ]:
def obj.exec(mapping):
    eval(mapping['f'] + "(self)")
    raw_data(self, mapping)
    
